In [1]:
import pandas as pd # library for data analsysis

import json # library to handle JSON files

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors # Matplotlib and associated plotting modules

from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # website scraping libraries and packages in Python from BeautifulSoup 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                

In [20]:
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.10.1              |             py_0          59 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         112 KB

The following NEW packages will be INSTALLED:

    branca:  0.4.0-py_0  conda-forge
    folium:  0.10.1-py_0 conda-forge
    vincent: 0.4.4-py_1  conda-forge


folium-0.10.1        | 59 KB     | ##################################### | 100% 
vincent-0.4.4        | 28 KB     | ##################################### | 100% 
branca-0.4.0         | 26 KB   

In [31]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [69]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [70]:
soup = BeautifulSoup(data, 'html.parser')

In [71]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [72]:
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [73]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [74]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [75]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [76]:
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [77]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [78]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


In [88]:
toronto_df_grouped.shape

(103, 3)

In [89]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [90]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [91]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [92]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_new[toronto_df_new["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,Wexford / Maryvale,43.750072,-79.295849
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...,43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420


In [93]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [94]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

NameError: name 'folium' is not defined

In [95]:
CLIENT_ID = 'FMHTSXNZPPEZGY12OJJ0WB0QHSEG5WSU0YSNSPSKQ2LYEITT' # your Foursquare ID
CLIENT_SECRET = 'GOL0IITGSVCYUYR2WKNGNGY5WMDTREQ4UX1LMIWCIOC1ASJN'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FMHTSXNZPPEZGY12OJJ0WB0QHSEG5WSU0YSNSPSKQ2LYEITT
CLIENT_SECRET:GOL0IITGSVCYUYR2WKNGNGY5WMDTREQ4UX1LMIWCIOC1ASJN


In [97]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], 
                                                  toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id=FMHTSXNZPPEZGY12OJJ0WB0QHSEG5WSU0YSNSPSKQ2LYEITT&client_secret=GOL0IITGSVCYUYR2WKNGNGY5WMDTREQ4UX1LMIWCIOC1ASJN&v=20180605 \
     &ll=43.653963,-79.387207&radius=500&limit=100".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [98]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4429, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,Textile Museum of Canada,43.654396,-79.386500,Art Museum
2,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,Cafe Plenty,43.654571,-79.389450,Café
3,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,Sansotei Ramen 三草亭,43.655157,-79.386501,Ramen Restaurant
4,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,Japango,43.655268,-79.385165,Sushi Restaurant


In [99]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M1B,Scarborough,Malvern / Rouge,43,43,43,43,43,43
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43,43,43,43,43,43
M1E,Scarborough,Guildwood / Morningside / West Hill,43,43,43,43,43,43
M1G,Scarborough,Woburn,43,43,43,43,43,43
M1H,Scarborough,Cedarbrae,43,43,43,43,43,43
M1J,Scarborough,Scarborough Village,43,43,43,43,43,43
M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43,43,43,43,43,43
M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43,43,43,43,43,43
M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43,43,43,43,43,43


In [100]:
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(4429, 36)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,...,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M1B,Scarborough,Malvern / Rouge,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1B,Scarborough,Malvern / Rouge,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1B,Scarborough,Malvern / Rouge,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1B,Scarborough,Malvern / Rouge,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,M1B,Scarborough,Malvern / Rouge,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [103]:
import numpy as np

In [101]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(103, 36)


,PostalCode,Borough,Neighborhoods,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Bar,Breakfast Spot,Bubble Tea Shop,...,Plaza,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Smoke Shop,Sushi Restaurant,Tapas Restaurant,University,Vegetarian / Vegan Restaurant
0,M1B,Scarborough,Malvern / Rouge,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
2,M1E,Scarborough,Guildwood / Morningside / West Hill,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
3,M1G,Scarborough,Woburn,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
4,M1H,Scarborough,Cedarbrae,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
5,M1J,Scarborough,Scarborough Village,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256
9,M1N,Scarborough,Birch Cliff / Cliffside West,0.023256,0.069767,0.023256,0.023256,0.023256,0.046512,0.023256,...,0.023256,0.023256,0.023256,0.023256,0.023256,0.023256,0.046512,0.023256,0.023256,0.023256


In [104]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(103, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
2,M1E,Scarborough,Guildwood / Morningside / West Hill,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
3,M1G,Scarborough,Woburn,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
4,M1H,Scarborough,Cedarbrae,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
5,M1J,Scarborough,Scarborough Village,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
9,M1N,Scarborough,Birch Cliff / Cliffside West,Coffee Shop,Art Gallery,Café,Sushi Restaurant,Breakfast Spot,Japanese Restaurant,Vegetarian / Vegan Restaurant,Donut Shop,Gift Shop,Gastropub
